In [2]:
import pandas as pd 
import pickle
import time
import py2neo
import urllib3
import os.path
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

service_df = pd.read_pickle('../pickles/services/pools.pickle')
x = service_df['last used in block'].value_counts()
x = x.to_frame()
x = x.iloc[:50]
block_list = set(x.index)
block_list = sorted(block_list)

In [ ]:
def query_database(query):
    # REMEMBER TO BE CONNECTED TO IMPERIAL WIFI!
    graph_db = py2neo.Graph("https://dsi-bitcoin.doc.ic.ac.uk:7473/db/data/", auth=("adi", "aditi123"))
    return graph_db.run(query)

def get_block_data(blk):
    query_string = """
                    MATCH (b:Block) <-[:MINED_IN]- (t:Tx) <-[:IN]- (txi:TxIn) <-[:UNLOCK]- (iadr:Address)
                    WHERE b.height = {}
                    MATCH (txi) <-[:SPENT]- (txo_in:TxOut) 
                    MATCH (oadr:Address) <-[:LOCK]- (txo_out:TxOut) <-[:OUT]- (t)
                    
                    RETURN iadr.address as iadr, oadr.address as oadr, txo_in.value as input_val, txo_out.value as output_val, ID(txo_in) as id_txo_in, ID(txi) as id_txi, ID(t) as id_t, ID(txo_out) as id_txo_out
                    """.format(blk)
    return query_string

def get_coinbase(blk):
    query_string = """
                    MATCH (b:Block) <-[:MINED_IN]- (t:Tx)<-[:IN]-(cb:CBscript)
                    MATCH (oadr:Address) <-[:LOCK]- (txo_out:TxOut) <-[:OUT]- (t)
                    WHERE b.height = {}
                    RETURN                       oadr.address as oadr,                            txo_out.value as output_val,                          ID(cb) as id_txi, ID(t) as id_t, ID(txo_out) as id_txo_out             
                    """.format(blk)
    return query_string

def check_block(first_block, last_block):
    query_string = """ 
                    MATCH (b:Block) 
                    WHERE b.height >= {} AND b.height <= {}
                    RETURN b
                    """.format(first_block, last_block)
    return query_string

def write_to_csv(result,string):

    df = result.to_data_frame()

    if (df.empty):
        print("Something went wrong, there is no data for this/these blocks")
    else:
        df.to_csv('{}.csv'.format(string), encoding='utf-8', index=False)
        
def seen_before_out(block, adr):
    params = {'block':block,'adr':adr}
    query_string = """
                    MATCH (a:Address) <-[:LOCK]- (to:TxOut) <-[:OUT]- (t1:Tx)-[:MINED_IN]->(b1:Block)
                    WHERE a.address = "{adr}" AND b1.height<{block}
                    RETURN b1.height LIMIT 1
                    """.format(**params)
    
    data_fr = query_database(query_string).to_data_frame()
    if data_fr.empty:
        return False
    return True

def seen_before_in(block, adr):
    params = {'block':block,'adr':adr}
    query_string = """
                    MATCH (a:Address) -[:UNLOCK]-> (ti:TxIn) -[:IN]-> (t2:Tx)-[:MINED_IN]->(b2:Block)
                    WHERE a.address = "{adr}" AND b2.height<{block}
                    RETURN b2.height LIMIT 1
                    """.format(**params)
    data_fr = query_database(query_string).to_data_frame()
    if data_fr.empty:
        return False
    return True

#temptime = time.time()
#starttime = time.time()

def all_block_data(block):
#for block in range(first_block,last_block+1,1):
    if(os.path.exists("../pickles/df/{}.pickle".format(block))):
        print('df already exists')
        df = pd.read_pickle("../pickles/df/{}.pickle".format(block))
        print('Got all block data')
        return df
    
    result = query_database(get_block_data(block))
    df = result.to_data_frame()
    
    #Append coinbase txs to dataframe
    result_cb = query_database(get_coinbase(block))
    df_cb = result_cb.to_data_frame()
    df_cb.insert(0, 'iadr', '0')
    df_cb.insert(3, 'id_txo_in', 0)
    df_cb.insert(5, 'input_val', 0)
    
    df = df.append(df_cb)
    df = df.reset_index()
    df = df.drop(columns=['index'])
    df.to_pickle("../pickles/df/{}.pickle".format(block))
    print('Got all block data')
    return df
    #print(block, " processed in ", time.time()-temptime)
    #temptime = time.time()  
#print("Total time to process everything: {}".format(time.time()-starttime))


initaltime = time.time()
for block in block_list:
#for block in range(first_block,last_block+1,1):
    starttime = time.time()
    print(block)
    if(os.path.exists('../pickles/otc/otc_{}.pickle'.format(block))):
        print('otc already exists')
        continue
    
    df = all_block_data(block)
    
    if df.empty:
        print("Block {} contained nothing".format(block))
        continue

    addresses = list(df.oadr.value_counts().index)

    not_seen =[]
    import time

    temptime = time.time()

    for adr in addresses:
        if seen_before_in(block, adr):
            continue
        elif seen_before_out(block, adr):
            continue 
        else:
            not_seen.append(adr)
        #print(adr, " processed in ", time.time()-temptime)
        temptime = time.time()
    print('otc done')
    
    with open('../pickles/otc/out.txt', 'a') as f:
        print("Total time to process block {}: {}".format(block, time.time()-starttime), file=f)
        
    with open('../pickles/otc/otc_{}.pickle'.format(block),'wb') as f:
        pickle.dump(not_seen,f)

with open('../pickles/otc/out.txt', 'a') as f:
    print("Total time to process everything: {}".format(time.time()-initaltime), file=f)

379568
df already exists
Got all block data
otc done
383891
Got all block data
otc done
384065
Got all block data
otc done
384068
Got all block data
otc done
385568
Got all block data
otc done
385571
Got all block data


In [ ]:
myNotebook = "./pickle_to_df.ipynb"

%run $myNotebook

In [ ]:
# addresses1 = list(df.oadr.value_counts()[df.oadr.value_counts()>1].index)
# addresses2 = list(df.oadr.value_counts()[df.oadr.value_counts()==1].index)

In [ ]:
# not_seen = [adr for adr in addresses1[:5] if not seen_before_in(block, adr) and not seen_before_out(block, adr)]